# High-level MXNet Example

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from common.params import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]
Numpy:  1.13.3
MXNet:  0.11.0
GPU:  ['Tesla K80']


In [3]:
def create_symbol():
    data = mx.symbol.Variable('data')
    # size = [(old-size - kernel + 2*padding)/stride]+1
    # if kernel = 3, pad with 1 either side
    conv1 = mx.symbol.Convolution(data=data, num_filter=50, pad=(1,1), kernel=(3,3))
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    conv2 = mx.symbol.Convolution(data=relu1, num_filter=50, pad=(1,1), kernel=(3,3))
    pool1 = mx.symbol.Pooling(data=conv2, pool_type="max", kernel=(2,2), stride=(2,2))
    relu2 = mx.symbol.Activation(data=pool1, act_type="relu")
    drop1 = mx.symbol.Dropout(data=relu2, p=0.25)
    
    conv3 = mx.symbol.Convolution(data=drop1, num_filter=100, pad=(1,1), kernel=(3,3))
    relu3 = mx.symbol.Activation(data=conv3, act_type="relu")
    conv4 = mx.symbol.Convolution(data=relu3, num_filter=100, pad=(1,1), kernel=(3,3))
    pool2 = mx.symbol.Pooling(data=conv4, pool_type="max", kernel=(2,2), stride=(2,2))
    relu4 = mx.symbol.Activation(data=pool2, act_type="relu")
    drop2 = mx.symbol.Dropout(data=relu4, p=0.25)
           
    flat1 = mx.symbol.Flatten(data=drop2)
    fc1 = mx.symbol.FullyConnected(data=flat1, num_hidden=512)
    relu7 = mx.symbol.Activation(data=fc1, act_type="relu")
    drop4 = mx.symbol.Dropout(data=relu7, p=0.5)
    fc2 = mx.symbol.FullyConnected(data=drop4, num_hidden=N_CLASSES) 
    
    input_y = mx.symbol.Variable('softmax_label')  
    m = mx.symbol.SoftmaxOutput(data=fc2, label=input_y, name="softmax")
    return m

In [4]:
def init_model(m):
    if GPU:
        ctx = [mx.gpu(0)]
    else:
        ctx = mx.cpu()
    mod = mx.mod.Module(context=ctx, symbol=m)
    mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
    # Glorot-uniform initializer
    mod.init_params(initializer=mx.init.Xavier(rnd_type='uniform'))
    mod.init_optimizer(optimizer='sgd', 
                       optimizer_params=(('learning_rate', LR), ('momentum', MOMENTUM), ))
    return mod

In [5]:
%%time
# Data into format for library
#x_train, x_test, y_train, y_test = mnist_for_library(channel_first=True)
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
# Load data-iterator
train_iter = mx.io.NDArrayIter(x_train, y_train, BATCHSIZE, shuffle=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 1.25 s, sys: 1.04 s, total: 2.29 s
Wall time: 2.29 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 1.62 ms, sys: 936 µs, total: 2.56 ms
Wall time: 1.85 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 935 ms, sys: 769 ms, total: 1.7 s
Wall time: 2.23 s


In [8]:
%%time
# 144s
# Train and log accuracy
metric = mx.metric.create('acc')
for j in range(EPOCHS):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        model.forward(batch, is_train=True) 
        model.update_metric(metric, batch.label)
        model.backward()              
        model.update()
    print('Epoch %d, Training %s' % (j, metric.get()))

Epoch 0, Training ('accuracy', 0.32448849104859334)
Epoch 1, Training ('accuracy', 0.49570412404092073)
Epoch 2, Training ('accuracy', 0.58082241048593353)
Epoch 3, Training ('accuracy', 0.63898657289002558)
Epoch 4, Training ('accuracy', 0.68156569693094626)
Epoch 5, Training ('accuracy', 0.71653212915601028)
Epoch 6, Training ('accuracy', 0.7433064258312021)
Epoch 7, Training ('accuracy', 0.76380674552429673)
Epoch 8, Training ('accuracy', 0.78790361253196928)
Epoch 9, Training ('accuracy', 0.80189018542199486)
CPU times: user 1min 41s, sys: 1min 10s, total: 2min 51s
Wall time: 2min 39s


In [9]:
%%time
y_guess = model.predict(mx.io.NDArrayIter(x_test, batch_size=BATCHSIZE, shuffle=False))
y_guess = np.argmax(y_guess.asnumpy(), axis=-1)

CPU times: user 744 ms, sys: 609 ms, total: 1.35 s
Wall time: 1.03 s


In [10]:
print("Accuracy: ", sum(y_guess == y_test)/len(y_guess))

Accuracy:  0.7577
